In [18]:
import uproot

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Print the structure of the trees
print("Hits Tree Structure:")
hits_tree.show()  # Displays the available branches and their types

print("\nSource Tree Structure:")
source_tree.show()  # Displays the available branches and their types


Hits Tree Structure:
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
PDGEncoding          | int32_t                  | AsDtype('>i4')
trackID              | int32_t                  | AsDtype('>i4')
parentID             | int32_t                  | AsDtype('>i4')
trackLocalTime       | double                   | AsDtype('>f8')
time                 | double                   | AsDtype('>f8')
runID                | int32_t                  | AsDtype('>i4')
eventID              | int32_t                  | AsDtype('>i4')
sourceID             | int32_t                  | AsDtype('>i4')
primaryID            | int32_t                  | AsDtype('>i4')
posX                 | float                    | AsDtype('>f4')
posY                 | float                    | AsDtype('>f4')
posZ                 | float                    | AsDtype('>f4')
localPosX            | float         

In [32]:
for i in range(0,10):
    # Load specific volumeID (e.g., volumeID[4]) and positions to visualize potential collimator structures
    filtered_hits_data = hits_tree.arrays(["posX", "posY", f"volumeID[{i}]"], library="pd")

    # Plot each unique volumeID[4] value in a different color to explore potential collimator structures
    plt.figure(figsize=(10, 8))
    for value in filtered_hits_data[f"volumeID[{i}]"].unique():
        subset = filtered_hits_data[filtered_hits_data[f"volumeID[{i}]"] == value]
        plt.scatter(subset["posX"], subset["posY"], label=f"volumeID[{i}] = {value}", alpha=0.6, s=20)

    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.title(f"Positions by volumeID[{i}] Values")
    plt.legend()

    # Save the plot
    plot_path = f"collimator_positions_by_volumeID{i}.png"
    plt.savefig(plot_path)
    plt.show()

    print(f"Plot saved as {plot_path}")


Plot saved as collimator_positions_by_volumeID0.png
Plot saved as collimator_positions_by_volumeID1.png
Plot saved as collimator_positions_by_volumeID2.png
Plot saved as collimator_positions_by_volumeID3.png
Plot saved as collimator_positions_by_volumeID4.png
Plot saved as collimator_positions_by_volumeID5.png
Plot saved as collimator_positions_by_volumeID6.png
Plot saved as collimator_positions_by_volumeID7.png
Plot saved as collimator_positions_by_volumeID8.png
Plot saved as collimator_positions_by_volumeID9.png


In [16]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "volumeID[2]", "volumeID[6]", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_volumeID_2 = 0  # Replace with the desired volumeID[2] value
selected_volumeID_6 = 0  # Replace with the desired volumeID[6] value

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["volumeID[2]"] == selected_volumeID_2) &
    (hits_data["volumeID[6]"] == selected_volumeID_6)
][["posX", "posY", "eventID"]]

# Ensure the detector positions are unique and pick the first one
selected_posX = selected_detector_positions["posX"].iloc[0]
selected_posY = selected_detector_positions["posY"].iloc[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Group by source positions and count occurrences
source_counts = matched_events.groupby(["sourcePosX", "sourcePosY"]).size().reset_index(name="count")

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.2, label="Source Positions")

# Plot all detector positions
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=2, alpha=1, label="Detector Positions")

# Highlight the selected detector
plt.scatter(selected_posX, selected_posY, c='red', s=10, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector, with count size
plt.scatter(source_counts["sourcePosX"], source_counts["sourcePosY"], c='purple', s=2, alpha=0.7, label="Grouped Sources Reaching Detector")

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Source and Detector Positions with Selected Detector Highlighted by volumeID[2] and volumeID[6]")
plt.legend()

# Save the plot
plot_path = "source_and_detector_positions_with_highlight_volumeID2_volumeID6.png"
plt.savefig(plot_path)
plt.show()
print(source_positions.describe())
print(source_positions["sourcePosX"].unique())
print(source_positions["sourcePosY"].unique())


          sourcePosX     sourcePosY        eventID
count  124716.000000  124716.000000  124716.000000
mean        4.108643       0.131716   18448.366577
std        35.853565      32.563976   10656.976904
min       -62.498875     -62.498741       0.000000
25%       -26.090950     -25.799936    9214.000000
50%         5.867578       0.041295   18460.000000
75%        35.516204      26.248422   27660.000000
max        62.499516      62.496105   37168.000000
[  2.4729073 -22.174835   -6.7370386 ... -18.737719   39.37371
 -37.133926 ]
[ 35.326504   38.58129     1.8649846 ... -24.387278  -23.589924
   6.9210234]


In [46]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "volumeID[2]", "volumeID[6]", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_volumeID_2 = 0  # Replace with the desired volumeID[2] value
selected_volumeID_6 = 0  # Replace with the desired volumeID[6] value

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["volumeID[2]"] == selected_volumeID_2) &
    (hits_data["volumeID[6]"] == selected_volumeID_6)
][["posX", "posY", "eventID"]]

# Ensure the detector positions are unique and pick the first one
selected_posX = selected_detector_positions["posX"].iloc[0]
selected_posY = selected_detector_positions["posY"].iloc[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Group by source positions and count occurrences
source_counts = matched_events.groupby(["sourcePosX", "sourcePosY"]).size().reset_index(name="count")

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.2, label="Source Positions")

# Plot all detector positions
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=2, alpha=1, label="Detector Positions")

# Highlight the selected detector
plt.scatter(selected_posX, selected_posY, c='red', s=10, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector, with count size
plt.scatter(source_counts["sourcePosX"], source_counts["sourcePosY"], c='purple', s=2, alpha=0.7, label="Grouped Sources Reaching Detector")

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Source and Detector Positions with Selected Detector Highlighted by volumeID[2] and volumeID[6]")

# Set equal scaling
plt.axis('equal')
plt.grid(True, which='both', axis='both', linestyle='--', color='gray', linewidth=0.5)


# Add legend
plt.legend()

# Save the plot
plot_path = "source_and_detector_positions_with_highlight_volumeID2_volumeID6.png"
plt.savefig(plot_path)
plt.show()

# Print some basic statistics
print(source_positions.describe())
print(source_positions["sourcePosX"].unique())
print(source_positions["sourcePosY"].unique())


         sourcePosX    sourcePosY        eventID
count  30405.000000  30405.000000   30405.000000
mean    -180.836014      0.268541   48204.453248
std      141.654007    107.328400   29924.516790
min     -437.497375   -249.985764       1.000000
25%     -301.588654    -72.260551   22905.000000
50%     -177.532303      0.189218   46161.000000
75%      -58.410690     74.242035   69594.000000
max       62.473713    249.974319  111686.000000
[ -83.6761   -190.4024    -66.30097  ...   21.717815   15.851701
 -280.91928 ]
[ -41.722786   109.14217   -142.77626   ...  -66.749054     4.9394007
  -36.44242  ]


In [27]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "volumeID[2]", "volumeID[6]", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_volumeID_2 = 3  # Replace with the desired volumeID[2] value
selected_volumeID_6 = 115  # Replace with the desired volumeID[6] value

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["volumeID[2]"] == selected_volumeID_2) &
    (hits_data["volumeID[6]"] == selected_volumeID_6)
][["posX", "posY", "eventID"]]

# Ensure the detector positions are unique and pick the first one
if selected_detector_positions.empty:
    raise ValueError("No detector positions found matching the specified criteria.")

selected_posX = selected_detector_positions["posX"].iloc[0]
selected_posY = selected_detector_positions["posY"].iloc[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Group by source positions and count occurrences
source_counts = matched_events.groupby(["sourcePosX", "sourcePosY"]).size().reset_index(name="count")

# Normalize the counts to get a probability density
source_counts["normalized_count"] = source_counts["count"] / source_counts["count"].sum()

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.2, label="Source Positions")

# Plot all detector positions
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=2, alpha=1, label="Detector Positions")

# Highlight the selected detector
plt.scatter(selected_posX, selected_posY, c='red', s=10, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector, with normalized count size
plt.scatter(
    source_counts["sourcePosX"], 
    source_counts["sourcePosY"], 
    c='purple', 
    s=2,  
    alpha=0.7, 
    label="Grouped Sources (Normalized)"
)

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("PPDF PLOT")
plt.legend()

# Save the plot
plot_path = "PPDF_PLOT.png"
plt.savefig(plot_path)
plt.show()


In [35]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "volumeID[2]", "volumeID[6]", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_volumeID_2 = 1  # Replace with the desired volumeID[2] value
selected_volumeID_6 = 120  # Replace with the desired volumeID[6] value

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["volumeID[2]"] == selected_volumeID_2) &
    (hits_data["volumeID[6]"] == selected_volumeID_6)
][["posX", "posY", "eventID"]]

# Ensure the detector positions are unique and pick the first one
selected_posX = selected_detector_positions["posX"].iloc[0]
selected_posY = selected_detector_positions["posY"].iloc[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Group by source positions and count occurrences
source_counts = matched_events.groupby(["sourcePosX", "sourcePosY"]).size().reset_index(name="count")

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions with equal size
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.2, label="Source Positions")

# Plot all detector positions with equal size
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=20, alpha=0.7, label="Detector Positions")

# Highlight the selected detector with equal size
plt.scatter(selected_posX, selected_posY, c='red', s=100, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector with fixed size
plt.scatter(
    source_counts["sourcePosX"],
    source_counts["sourcePosY"],
    c='purple',
    s=30,  # Fixed size for all grouped points
    alpha=0.7,
    label="Grouped Sources Reaching Detector"
)

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Source and Detector Positions with Equal Marker Sizes")
plt.legend()

# Save the plot
plot_path = "source_and_detector_positions_equal_marker_sizes.png"
plt.savefig(plot_path)
plt.show()


In [36]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./hits.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./singles.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "volumeID[2]", "volumeID[6]", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_volumeID_2 = 1  # Replace with the desired volumeID[2] value
selected_volumeID_6 = 120  # Replace with the desired volumeID[6] value

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["volumeID[2]"] == selected_volumeID_2) &
    (hits_data["volumeID[6]"] == selected_volumeID_6)
][["posX", "posY", "eventID"]]

# Ensure the detector positions are unique and pick the first one
selected_posX = selected_detector_positions["posX"].iloc[0]
selected_posY = selected_detector_positions["posY"].iloc[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Group by source positions and count occurrences
source_counts = matched_events.groupby(["sourcePosX", "sourcePosY"]).size().reset_index(name="count")

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.2, label="Source Positions")

# Plot all detector positions with a fixed size
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=30, alpha=0.7, label="Detector Positions")

# Highlight the selected detector with a larger size for emphasis
plt.scatter(selected_posX, selected_posY, c='red', s=100, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector, with count size
plt.scatter(
    source_counts["sourcePosX"],
    source_counts["sourcePosY"],
    c='purple',
    s=30,  # Fixed size for all grouped points
    alpha=0.7,
    label="Grouped Sources Reaching Detector"
)

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Source and Detector Positions with Uniform Detector Marker Sizes")
plt.legend()

# Save the plot
plot_path = "source_and_detector_positions_uniform_detector_sizes.png"
plt.savefig(plot_path)
plt.show()


In [4]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

# Load the ROOT files (replace the file paths with your actual file paths)
hits_file = uproot.open("./SPEBT.hits_merged.root")  # Replace with the correct path to your hits ROOT file
source_file = uproot.open("./SPEBT.Singles_merged.root")  # Replace with the correct path to your source ROOT file

# Load the trees
hits_tree = hits_file["tree;1"]
source_tree = source_file["tree;1"]

# Extract the relevant data from the trees
hits_data = hits_tree.arrays(["posX", "posY", "gantryID", "rsectorID", "moduleID", "submoduleID", "crystalID", "eventID"], library="pd")
source_data = source_tree.arrays(["sourcePosX", "sourcePosY", "sourcePosZ", "eventID"], library="pd")

# Extract the source and detector positions
source_positions = source_data[["sourcePosX", "sourcePosY", "eventID"]]
detector_positions = hits_data[["posX", "posY"]]

# Define selected detector properties
selected_detector = {
    "gantryID": 0,
    "rsectorID": 2,
    "moduleID": 0,
    "submoduleID": 1,
    "crystalID": 3
}

# Extract the selected detector's position
selected_detector_positions = hits_data[
    (hits_data["gantryID"] == selected_detector["gantryID"]) &
    (hits_data["rsectorID"] == selected_detector["rsectorID"]) &
    (hits_data["moduleID"] == selected_detector["moduleID"]) &
    (hits_data["submoduleID"] == selected_detector["submoduleID"]) &
    (hits_data["crystalID"] == selected_detector["crystalID"])
][["posX", "posY", "eventID"]]

selected_posX = selected_detector_positions["posX"].values[0]
selected_posY = selected_detector_positions["posY"].values[0]

# Find source events that reach the selected detector
matched_events = source_positions[source_positions["eventID"].isin(selected_detector_positions["eventID"])]

# Create ray endpoints based on source and detector positions for density calculation
ray_endpoints = []
for _, row in matched_events.iterrows():
    # Linear interpolation of points between source and detector
    x_points = np.linspace(row["sourcePosX"], selected_posX, 100)
    y_points = np.linspace(row["sourcePosY"], selected_posY, 100)
    ray_endpoints.append(np.vstack((x_points, y_points)))

# Flatten arrays for density calculation
ray_endpoints = np.hstack(ray_endpoints)
x_ray = ray_endpoints[0]
y_ray = ray_endpoints[1]

# Calculate the density of rays
kde = gaussian_kde([x_ray, y_ray])
density = kde([x_ray, y_ray])

# Plotting
plt.figure(figsize=(10, 8))

# Plot all source positions
plt.scatter(source_positions["sourcePosX"], source_positions["sourcePosY"], c='green', s=10, alpha=0.5, label="Source Positions")

# Plot all detector positions
plt.scatter(detector_positions["posX"], detector_positions["posY"], c='blue', s=20, alpha=0.7, label="Detector Positions")

# Highlight the selected detector
plt.scatter(selected_posX, selected_posY, c='red', s=100, label="Selected Detector", edgecolors='black')

# Plot grouped source positions that reach the selected detector, with count size
plt.scatter(matched_events["sourcePosX"], matched_events["sourcePosY"], c='purple', s=20, alpha=0.7, label="Grouped Sources Reaching Detector")

# Plot the density heatmap of rays
plt.scatter(x_ray, y_ray, c=density, cmap="hot", s=5, alpha=0.5, label="Ray Density Heatmap")

# Labeling axes
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Source and Detector Positions with Ray Density Heatmap")
plt.legend()

# Save the plot
plot_path = "source_and_detector_positions_with_ray_density_heatmap.png"
plt.savefig(plot_path)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/user/tridevme/Tridev_SPEBT/NewConfiguration/GATE-Macro/SPEBT.hits_merged.root'

In [15]:
import uproot
import pandas as pd

# Open the ROOT file for xlayer5 (layerID = 4)
file = uproot.open("hits.root")
hits_tree = file["tree;1"]

# Load the branches into a DataFrame
data = hits_tree.arrays(library="pd", filter_name=["layerID", "volumeID[2]", "posX", "posY", "posZ", "edep"])

# Filter for layerID = 4 (xlayer5) and volumeID = 2 (third crystal)
selected_hits = data.query("layerID==")

print(selected_hits)


Empty DataFrame
Columns: [posX, posY, posZ, edep, layerID]
Index: []
